In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import undetected_chromedriver as uc
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import os
import time

from crypto_web_scraping.scrap import *

In [4]:
driver = create_driver()
df = get_basic_info(driver)

In [ ]:
df

In [ ]:
from selenium import webdriver
url = “https://finance.yahoo.com/topic/stock-market-news"
links =[]
for i in range(1,10):
    #define a driver
    driver = webdriver.Chrome(‘C:\chromedriver’) #use the path in your computer
    driver.implicitly_wait(5)
    #get request
    driver.get(url)
    #keep clicking each news article and grab their url
    elem = driver.find_element_by_css_selector(“#Fin-Stream > ul > li:nth-child({})”.format(i))
    #get the URL after click
    elem.click()
    #store in a list
    links.append(driver.current_url)

In [11]:
url = "https://finance.yahoo.com/topic/stock-market-news"
driver = create_driver(url)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="consent-page"]/div/div/div/form/div[2]/div[2]/button[1]'))).click()
driver.title

'Latest Stock Market News'

In [46]:
url = 'https://finance.yahoo.com/quote/TSLA?p=TSLA&.tsrc=fin-srch'
driver = create_driver(url)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="consent-page"]/div/div/div/form/div[2]/div[2]/button[1]'))).click()

driver.implicitly_wait(15)

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [47]:
#driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
articles = driver.find_elements(By.CLASS_NAME, 'js-stream-content.Pos\(r\)')
len(articles)

3

In [24]:
articles = driver.find_elements(By.CLASS_NAME, 'Ov\(h\).Pend\(44px\).Pstart\(25px\)')
len(articles)

9